## We will now do a clustering analysis using both datasets.

## First we import of the utilities

In [123]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"
from jupyterthemes import jtplot
jtplot.style(theme='onedork')
%matplotlib notebook
#%matplotlib inline
import folium
from geopy.geocoders import Nominatim
import json
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
print("Libraries Imported")

Libraries Imported


## Now we read the datasets we need

In [2]:
df_toronto=pd.read_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\Toronto_Venues_800m.csv",index_col=0)
df_new_york=pd.read_csv("C:\\Users\\...\\Documents\\Data_Analysis_Courses\\Data Science\\07_Capstone\\New_York_Venues_800m.csv",index_col=0)
print("Toronto Dataset")
print(df_toronto.shape)
print(df_toronto.columns)
print()
print("New York Dataset")
print(df_new_york.shape)
print(df_new_york.columns)

Toronto Dataset
(9251, 7)
Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

New York Dataset
(17095, 7)
Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')


#### We need now to merge the two datasets toghether

In [63]:
df_toronto['City']='Toronto'
df_new_york['City']='New York'
merged_dataset=df_toronto.append(df_new_york)
# print(merged_dataset.head())
# print(merged_dataset.tail())
cols=merged_dataset.columns
[cols[-1]]+list(cols[0:-1])
cols2=[cols[-1]]+list(cols[0:-1])
merged_dataset=merged_dataset[cols2]
print(merged_dataset.shape)
merged_dataset.head()
# merged_dataset.tail()

(26346, 8)


,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Toronto,Parkwoods,43.7588,-79.320197,Allwyn's Bakery,43.759840,-79.324719,Caribbean Restaurant
1,Toronto,Parkwoods,43.7588,-79.320197,LCBO,43.757774,-79.314257,Liquor Store
2,Toronto,Parkwoods,43.7588,-79.320197,Tim Hortons,43.760668,-79.326368,Café
3,Toronto,Parkwoods,43.7588,-79.320197,A&W Canada,43.760643,-79.326865,Fast Food Restaurant
4,Toronto,Parkwoods,43.7588,-79.320197,Dollarama,43.757317,-79.312578,Discount Store


## Clustering Analysis for Both Cities

### First we need to create once again the dataset with the top venues for each neighboorhood 

In [29]:
list(merged_dataset.columns[-2:]) + list(merged_dataset.columns[:-2])


['Venue Longitude',
 'Venue Category',
 'City',
 'Neighborhood',
 'Neighborhood Latitude',
 'Neighborhood Longitude',
 'Venue',
 'Venue Latitude']

In [107]:
Venues_hot = pd.get_dummies(merged_dataset[['Venue Category']], prefix="", prefix_sep="")
Venues_hot=Venues_hot.drop('Neighborhood', 1)
# add neighborhood column back to dataframe

Venues_hot['City'] = merged_dataset['City']
Venues_hot['Neighborhood'] = merged_dataset['Neighborhood']
# move neighborhood column to the first column
fixed_columns = list(Venues_hot.columns[-2:]) + list(Venues_hot.columns[:-2])

Venues_hot = Venues_hot[fixed_columns]

print(Venues_hot.shape)
Venues_hot.head()


(26346, 494)


,City,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Toronto,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Toronto,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Toronto,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Toronto,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Toronto,Parkwoods,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [108]:
Venues_hot_group = Venues_hot.groupby(['City','Neighborhood']).mean().reset_index()
print(Venues_hot_group.shape)
Venues_hot_group.head()

(508, 494)


,City,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,New York,Allerton,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,New York,Annadale,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,New York,Arden Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,New York,Arlington,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,New York,Arrochar,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [109]:
def return_most_common_venues_mod(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [110]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City','Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['City'] = Venues_hot_group['City']
neighborhoods_venues_sorted['Neighborhood'] = Venues_hot_group['Neighborhood']

for ind in np.arange(Venues_hot_group.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues_mod(Venues_hot_group.iloc[ind, :], num_top_venues)

print(neighborhoods_venues_sorted.shape)
neighborhoods_venues_sorted.tail()

(508, 7)


,City,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
503,Toronto,Woodbine Heights,Park,Pizza Place,Cosmetics Shop,Beer Store,Pharmacy
504,Toronto,York Mills,Recreation Center,Pet Store,Sports Club,Wine Shop,Tennis Court
505,Toronto,York Mills West,Coffee Shop,Gym / Fitness Center,Restaurant,Tennis Court,Business Service
506,Toronto,York University,Coffee Shop,Restaurant,Shopping Mall,Café,Pizza Place
507,Toronto,Yorkville,Café,Coffee Shop,Italian Restaurant,French Restaurant,Restaurant


In [189]:
# set number of clusters
kclusters = 5

Venues_clustering = Venues_hot_group.drop(['City','Neighborhood'], 1)

# run k-means clustering
Merge_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Venues_clustering)

# check cluster labels generated for each row in the dataframe
Merge_kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 1, 2, 2, 2])

In [190]:
df_merged_latlon=merged_dataset[['City','Neighborhood','Neighborhood Latitude','Neighborhood Longitude']].groupby(['City','Neighborhood']).mean()
print(df_merged_latlon.shape)

# add clustering labels
neighborhoods_venues_sorted=neighborhoods_venues_sorted.drop('Cluster Labels',1)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', Merge_kmeans.labels_)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
#Merged_result = df_merged_latlon.join(neighborhoods_venues_sorted, on=['City','Neighborhood'])
Merged_result=pd.merge(df_merged_latlon,neighborhoods_venues_sorted,on=['City','Neighborhood'])
print(Merged_result.shape)
Merged_result.head() # check the last columns!

(508, 2)
(508, 10)


,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,New York,Allerton,40.865788,-73.859319,2,Donut Shop,Pizza Place,Pharmacy,Sandwich Place,Deli / Bodega
1,New York,Annadale,40.538114,-74.178549,2,Pizza Place,Restaurant,Sports Bar,Trail,Train Station
2,New York,Arden Heights,40.549286,-74.185887,2,Mexican Restaurant,Food,Optical Shop,Shopping Mall,Elementary School
3,New York,Arlington,40.635325,-74.165104,2,Bus Stop,Deli / Bodega,Home Service,Boat or Ferry,Coffee Shop
4,New York,Arrochar,40.596313,-74.067124,2,Deli / Bodega,Bus Stop,Beach,Baseball Field,Liquor Store


In [193]:
Merged_result_NY=Merged_result[Merged_result["City"]=='New York']

latitude=40.7128
longitude=-74.0060
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Merged_result_NY['Neighborhood Latitude'], Merged_result_NY['Neighborhood Longitude'], Merged_result_NY['Neighborhood'], Merged_result_NY['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [194]:
Merged_result_TO=Merged_result[Merged_result["City"]=='Toronto']

latitude=43.6532
longitude=-79.3832
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11.2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Merged_result_TO['Neighborhood Latitude'], Merged_result_TO['Neighborhood Longitude'], Merged_result_TO['Neighborhood'], Merged_result_TO['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [217]:
Merged_result.loc[Merged_result['Cluster Labels'] == 0, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,New York,0,Playground,Home Service,Indian Restaurant,Beach,Athletics & Sports
20,New York,0,Beach,Spa,Deli / Bodega,Trail,Pharmacy
32,New York,0,Deli / Bodega,Chinese Restaurant,Park,Beach,Metro Station
125,New York,0,Beach,Surf Spot,Burger Joint,Donut Shop,Taco Place
173,New York,0,Baseball Field,Beach,Chinese Restaurant,Dessert Shop,Bagel Shop
187,New York,0,Beach,Park,Pizza Place,Bus Stop,Zoo Exhibit
235,New York,0,Beach,Pizza Place,Deli / Bodega,Ice Cream Shop,BBQ Joint
236,New York,0,Beach,Pizza Place,Deli / Bodega,Donut Shop,Ice Cream Shop
241,New York,0,Baseball Field,Theater,Beach,Boat or Ferry,Irish Pub
245,New York,0,Beach,Supermarket,Home Service,Park,Spa


In [184]:
Merged_result[Merged_result['4th Most Common Venue']=='Beach']

,City,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,New York,Arverne,40.589144,-73.791992,2,Surf Spot,Sandwich Place,Donut Shop,Beach,Bus Stop
15,New York,Bayswater,40.611322,-73.765968,0,Playground,Home Service,Indian Restaurant,Beach,Athletics & Sports
32,New York,Broad Channel,40.603027,-73.820055,0,Deli / Bodega,Chinese Restaurant,Park,Beach,Metro Station


In [218]:
Merged_result.loc[Merged_result['Cluster Labels'] == 1, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
6,New York,1,Bar,Greek Restaurant,Bakery,Pizza Place,Seafood Restaurant
10,New York,1,Coffee Shop,Park,Memorial Site,Hotel,Plaza
11,New York,1,Spa,Italian Restaurant,Pizza Place,Bar,Greek Restaurant
24,New York,1,Harbor / Marina,Gym,Baseball Field,Donut Shop,Athletics & Sports
26,New York,1,Boxing Gym,Department Store,Bagel Shop,Park,Baseball Field
27,New York,1,Bar,Coffee Shop,Bakery,Arts & Crafts Store,Dance Studio
31,New York,1,Beach,Bank,Eastern European Restaurant,Bakery,Sushi Restaurant
35,New York,1,Park,Yoga Studio,Italian Restaurant,Coffee Shop,Ice Cream Shop
39,New York,1,Bar,Coffee Shop,Mexican Restaurant,Pizza Place,Dive Bar
40,New York,1,Baseball Field,Convenience Store,Construction & Landscaping,Pool,Emergency Room


In [219]:
Merged_result.loc[Merged_result['Cluster Labels'] == 2, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,New York,2,Donut Shop,Pizza Place,Pharmacy,Sandwich Place,Deli / Bodega
1,New York,2,Pizza Place,Restaurant,Sports Bar,Trail,Train Station
2,New York,2,Mexican Restaurant,Food,Optical Shop,Shopping Mall,Elementary School
3,New York,2,Bus Stop,Deli / Bodega,Home Service,Boat or Ferry,Coffee Shop
4,New York,2,Deli / Bodega,Bus Stop,Beach,Baseball Field,Liquor Store
5,New York,2,Surf Spot,Sandwich Place,Donut Shop,Beach,Bus Stop
7,New York,2,Pizza Place,Bus Station,Italian Restaurant,Bakery,Rental Car Location
8,New York,2,Pizza Place,Café,Korean Restaurant,Greek Restaurant,Cosmetics Shop
9,New York,2,Pizza Place,Bank,Bakery,Chinese Restaurant,Japanese Restaurant
12,New York,2,Clothing Store,Donut Shop,Supermarket,Italian Restaurant,American Restaurant


In [220]:
Merged_result.loc[Merged_result['Cluster Labels'] == 3, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]].tail(20)

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
466,Toronto,3,Pool,Zoo Exhibit,Electronics Store,Elementary School,Emergency Room


In [221]:
Merged_result.loc[Merged_result['Cluster Labels'] == 4, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]].tail(20)

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
402,Toronto,4,Pharmacy,Park,Pizza Place,Convenience Store,Fast Food Restaurant
404,Toronto,4,Park,Bank,Flower Shop,Baseball Field,Golf Course
409,Toronto,4,Park,American Restaurant,Dessert Shop,Grocery Store,Flower Shop
410,Toronto,4,Park,American Restaurant,Dessert Shop,Grocery Store,Flower Shop
411,Toronto,4,Park,American Restaurant,Dessert Shop,Grocery Store,Flower Shop
413,Toronto,4,Park,Grocery Store,Gym / Fitness Center,Café,Playground
414,Toronto,4,Sandwich Place,Coffee Shop,Park,Convenience Store,Tennis Court
415,Toronto,4,Coffee Shop,Fast Food Restaurant,Furniture / Home Store,Tennis Court,Park
423,Toronto,4,Theater,Moving Target,Park,Zoo Exhibit,Fast Food Restaurant
425,Toronto,4,Café,Flower Shop,Beer Store,Coffee Shop,Pizza Place


In [161]:
Merged_result_TO=Merged_result[Merged_result["City"]=='Toronto']

latitude=43.6532
longitude=-79.3832
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Merged_result_TO['Neighborhood Latitude'], Merged_result_TO['Neighborhood Longitude'], Merged_result_TO['Neighborhood'], Merged_result_TO['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='Blue',
        fill=True,
        fill_color='Blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
Merged_result_NY=Merged_result[Merged_result["City"]=='New York']

latitude=40.7128
longitude=-74.0060
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Merged_result_NY['Neighborhood Latitude'], Merged_result_NY['Neighborhood Longitude'], Merged_result_NY['Neighborhood'], Merged_result_NY['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [85]:
print("Toronto Dataset")
print(df_toronto.shape)
print(df_toronto.columns)
print(len(np.unique(df_toronto['Venue Category'].values)))
print(len(np.unique(df_new_york['Venue Category'].values)))
# print("New York Dataset")
# print(df_new_york.shape)
# print(df_new_york.columns)

Toronto Dataset
(9251, 8)
Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category', 'City'],
      dtype='object')
353
462


In [88]:
a=set(np.unique(df_toronto['Venue Category'].values))
b=set(np.unique(df_new_york['Venue Category'].values))

In [90]:
len(a&b)

322

In [92]:
b-a

{'Adult Boutique',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Auditorium',
 'Australian Restaurant',
 'Austrian Restaurant',
 'Auto Garage',
 'Auto Workshop',
 'Bath House',
 'Bed & Breakfast',
 'Bike Rental / Bike Share',
 'Bike Trail',
 'Border Crossing',
 'Botanical Garden',
 'Boxing Gym',
 'Cambodian Restaurant',
 'Campground',
 'Car Wash',
 'Caucasian Restaurant',
 'Circus',
 'Club House',
 'College Academic Building',
 'College Basketball Court',
 'College Bookstore',
 'College Cafeteria',
 'College Quad',
 'Colombian Restaurant',
 'Convention Center',
 'Coworking Space',
 'Cruise',
 'Currency Exchange',
 'Czech Restaurant',
 'Daycare',
 'Distillery',
 'Drugstore',
 'Dry Cleaner',
 'Duty-free Shop',
 'Elementary School',
 'Emergency Room',
 'English Restaurant',
 'Event Service',
 'Exhibit',
 'Factory',
 'Film Studio',
 'Financial or Legal Service',
 'Food Stand',
 'Forest',
 'Garden Center',
 'Go Kart Track',
 'Government Building',
 'Gun Range',
 'Gymnastics Gym',
 'Hala

In [195]:
Merged_result.loc[Merged_result['Cluster Labels'] == 0, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]

,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
15,New York,0,Playground,Home Service,Indian Restaurant,Beach,Athletics & Sports
20,New York,0,Beach,Spa,Deli / Bodega,Trail,Pharmacy
32,New York,0,Deli / Bodega,Chinese Restaurant,Park,Beach,Metro Station
125,New York,0,Beach,Surf Spot,Burger Joint,Donut Shop,Taco Place
173,New York,0,Baseball Field,Beach,Chinese Restaurant,Dessert Shop,Bagel Shop
187,New York,0,Beach,Park,Pizza Place,Bus Stop,Zoo Exhibit
235,New York,0,Beach,Pizza Place,Deli / Bodega,Ice Cream Shop,BBQ Joint
236,New York,0,Beach,Pizza Place,Deli / Bodega,Donut Shop,Ice Cream Shop
241,New York,0,Baseball Field,Theater,Beach,Boat or Ferry,Irish Pub
245,New York,0,Beach,Supermarket,Home Service,Park,Spa


In [216]:
cluster0=Merged_result.loc[Merged_result['Cluster Labels'] == 0, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]
cluster1=Merged_result.loc[Merged_result['Cluster Labels'] == 1, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]
cluster2=Merged_result.loc[Merged_result['Cluster Labels'] == 2, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]
cluster3=Merged_result.loc[Merged_result['Cluster Labels'] == 3, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]
cluster4=Merged_result.loc[Merged_result['Cluster Labels'] == 4, Merged_result.columns[[0] + list(range(4, Merged_result.shape[1]))]]
cluster0.iloc[:,2].value_counts().index[0]
cluster0_val=[]
cluster1_val=[]
cluster2_val=[]
cluster3_val=[]
cluster4_val=[]
for i in range(2,7):
    cluster0_val.append(cluster0.iloc[:,i].value_counts().index[0])
    cluster1_val.append(cluster1.iloc[:,i].value_counts().index[0])
    cluster2_val.append(cluster2.iloc[:,i].value_counts().index[0])
    cluster3_val.append(cluster3.iloc[:,i].value_counts().index[0])
    cluster4_val.append(cluster4.iloc[:,i].value_counts().index[0])
print(cluster0_val)
print()
print(cluster1_val)
print()
print(cluster2_val)
print()
print(cluster3_val)
print()
print(cluster4_val)

['Beach', 'Beach', 'Deli / Bodega', 'Donut Shop', 'Athletics & Sports']

['Coffee Shop', 'Coffee Shop', 'Café', 'Pizza Place', 'Coffee Shop']

['Pizza Place', 'Pizza Place', 'Sandwich Place', 'Donut Shop', 'Deli / Bodega']

['Pool', 'Zoo Exhibit', 'Electronics Store', 'Elementary School', 'Emergency Room']

['Park', 'Park', 'Park', 'Park', 'Fast Food Restaurant']
